In [ ]:
pip install keras_tuner

In [ ]:
#Step 1: libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras_tuner import RandomSearch
import matplotlib.pyplot as plt

In [ ]:
#Step 2: Loading dataset and preprocess
(train_images, train_labels),(test_images, test_labels) = datasets.cifar10.load_data()

In [ ]:
#Step 3: Normalizing pixel values
train_images = train_images / 255.0
test_images = test_images/255.0

In [ ]:
#Step 4: Defining the class names
class_name = ['airplane','automobile','bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def build_model(hp):
  model = models.Sequential()
  model.add(layers.Conv2D(hp.Int('conv1_units',min_value=32,max_value=128,step=16),(3,3),activation='relu',input_shape=(32,32,3)))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(hp.Int('conv2_units',min_value=32,max_value=128,step=16),(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(hp.Int('conv3_units',min_value=32,max_value=128,step=16),(3,3),activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(hp.Int('dense_units',min_value=32,max_value=128,step=16),activation='relu'))
  model.add(layers.Dropout(hp.Float('dropout_rate',min_value=0.0,max_value=0.5,step=0.1)))
  model.add(layers.Dense(10,activation='softmax'))

  #Choose an optimizer and learning rate
  optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4]))

  model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

  return model




In [ ]:
#Step 5: Define the Tuner

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=15,
    executions_per_trial = 1,
    directory='my_dir',
    project_name='cifar10_tunning'
)

Reloading Tuner from my_dir/cifar10_tunning/tuner0.json


In [ ]:
#Step 6: Perform the Hyperparameter search
tuner.search(train_images,train_labels, epochs=5,validation_data=(test_images,test_labels))


Search: Running Trial #14

Value             |Best Value So Far |Hyperparameter
64                |48                |conv1_units
96                |112               |conv2_units
64                |96                |conv3_units
128               |80                |dense_units
0                 |0.3               |dropout_rate
0.0001            |0.001             |learning_rate



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.3001 - loss: 1.9164 - val_accuracy: 0.4771 - val_loss: 1.4521
Epoch 2/5
1332/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4857 - loss: 1.4359

In [ ]:
#Step 7: Get the best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

In [ ]:
#Step 8: Build the model with the best Hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images,train_labels,epochs=10,validation_data=(test_images,test_labels))

In [ ]:
#Step 9: Plotting the training and validation accuracy and loss value
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc='lower right')
plt.title('Train and Validation Accuracy')
plt.grid()

plt.subplot(1,2,2)
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim([0,1])
plt.legend(loc='upper right')
plt.title('Train and Validation Loss')
plt.grid()

plt.show()

In [ ]:
model.save("lenet5_model.h5")